In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
ds1radiotemp = pd.read_csv("../../dataset_1_radioTemperatures_20210303.csv",sep=';',dtype={'id_audit':np.str})

In [ ]:
ds2powerclass = pd.read_csv("../../dataset_2_powerClass_20210303.csv", sep=";", dtype={'id_audit':np.str})

In [ ]:
ds3radioid = pd.read_csv("../../dataset_3_radioId_20210303.csv",sep=";", dtype={'id_audit':np.str})

In [ ]:
ds1radiotemp.isnull().sum()

In [ ]:
ds1radiotemp.dropna(inplace=True)

In [ ]:
ds1radiotemp.info(verbose=True,null_counts=True)

In [ ]:
sensorData = ds1radiotemp.pivot_table(index=['id_audit'], columns=['field'],values=['value'])
sensorData = sensorData.droplevel(0, axis=1).reset_index()
sensorData.id_audit = sensorData.id_audit.astype(str)

In [ ]:
sensorData.info(verbose=True,null_counts=True)

In [ ]:
sensorData.drop(['TSensor_C1_','TSensor_D11_','TSensor_D12_','TSensor_D13_','TSensor_D14_','TSensor_D15_'
                ,'TSensor_D16_','TSensor_D17_','TSensor_D2_','TSensor_D3_','TSensor_D7_','TSensor_P39_'
                ,'TSensor_P40_','TSensor_P41_','TSensor_P42_','TSensor_P43_','TSensor_P44_','TSensor_P45_'
                ,'TSensor_P46_','TSensor_T63_','TSensor_T64_','TSensor_W73_','TSensor_W74_'], axis=1, inplace=True)

In [ ]:
sensorDataraw = sensorData.iloc[:,1:].values

In [ ]:
#from sklearn.preprocessing import StandardScaler
#stdscaler = StandardScaler()
#sensorDatascaled = stdscaler.fit_transform(sensorDataraw)

In [ ]:
#sensorDatascaled = np.insert(sensorDatascaled,0,sensorData.id_audit, axis=1)

In [ ]:
sensors = sensorData.columns.values[1:]
sensorDatascaled.shape,sensors

In [ ]:
sensorDatascaleddf = pd.DataFrame(data=sensorDatascaled,columns=sensors)

In [ ]:
sensorDataRed = sensorDatascaleddf.applymap(lambda x: np.nan if (x < -40 or x> 125) else x)
sensorDataRed['id_audit'] = sensorData.id_audit.values
sensorDataRed.dropna(inplace=True)
sensorDataRed.info(verbose=True,null_counts=True)

In [ ]:
sensorData.head()

In [ ]:
sensorDataRed.head()

In [ ]:
240 / sensorDataRed.shape[0]

In [ ]:
#from sklearn.ensemble import IsolationForest
#iso = IsolationForest(contamination=0.005)
from sklearn.cluster import DBSCAN
dbscan=DBSCAN()

In [ ]:
masks = []
for aSensor in sensors:
    yhat = dbscan.fit_predict(sensorDataRed[aSensor].values.reshape(-1, 1))
    mask = yhat == -1
    masks.append(mask)

In [ ]:
import pickle

In [ ]:
with open("masksData.dat", "wb") as fp:
    pickle.dump(masks,fp)

In [ ]:
with open("masksData.dat", "rb") as fp:
    masksloaded = pickle.load(fp)

In [ ]:
sensorDataRed.to_pickle("sensorDataRed.pkl")
sensorDataRedLoaded = pd.read_pickle("sensorDataRed.pkl")

In [ ]:
len(masksloaded)

In [ ]:
sensorDataRedLoaded.shape, sensorDataRed.shape

In [ ]:
maskpd = pd.DataFrame(data=masks, index=sensors)

In [ ]:
for mask in maskpd.values:
    sensorDataMasked = sensorDataRed[~mask]

In [ ]:
sensorDataMasked.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10,8)) 
sns.heatmap(sensorDataMasked.iloc[:,:-1].corr(), ax=ax)

In [ ]:
sensorDataMasked.iloc[:,:-1].corr()

In [ ]:
#c = sensorDataMasked.iloc[:,:-1].corr()
#s = c.unstack()
#so = s.sort_values(kind="quicksort").reset_index()

In [ ]:
corr_matrix = sensorDataMasked.iloc[:,:-1].corr()
#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))

In [ ]:
sol.head()

In [ ]:
sollist = sol.groupby(level=0).head(1)
sollist.tail()

In [ ]:
sollist2 = sollist.loc[sollist>0.9]

In [ ]:
sensorDataMasked.head()

In [ ]:
from sklearn.linear_model import LinearRegression
lnclf1 = LinearRegression()
lnclf2 = LinearRegression()

In [ ]:
maskpd.loc['TSensor_R47_'].value_counts()

In [ ]:
sensorDataRed.loc[maskpd.loc['TSensor_R47_'].values]['TSensor_R47_'].shape

In [ ]:
sollist2[0], sollist2.index[0]

In [ ]:
index = sollist2.index[0]

In [ ]:
yhat = iso.fit_predict(sensorDataRed.TSensor_R47_.values.reshape(-1, 1))
mask = yhat == -1
np.bincount(mask), len(mask)

In [ ]:
maskpd.loc[index[0]].value_counts()

In [ ]:
lnclf1.fit(sensorDataMasked[index[0]].values.reshape(-1, 1), sensorDataMasked[index[1]].values.reshape(-1, 1))

In [ ]:
ypred1 = lnclf1.predict(sensorDataRed.loc[maskpd.loc[index[0]].values][index[0]].values.reshape(-1, 1))

In [ ]:
residuals1 = sensorDataRed.loc[maskpd.loc[index[0]].values][index[1]].values.reshape(-1, 1)-ypred1

In [ ]:
plt.plot(sensorDataRed.loc[maskpd.loc[index[0]].values][index[0]].values.reshape(-1, 1),residuals1, 'o', color='darkblue')
plt.title("Residual Plot")
plt.xlabel("Last sensor")
plt.ylabel("Residual")

In [ ]:
for index, value in (sollist2.items()):
    #print(index[0], index[1],value)
    lnclf1.fit(sensorDataMasked[index[0]].values.reshape(-1, 1), sensorDataMasked[index[1]].values.reshape(-1, 1))
    ypred1 = lnclf1.predict(sensorDataRed.loc[maskpd.loc[index[0]].values][index[0]].values.reshape(-1, 1))
    residuals1 = sensorDataRed.loc[maskpd.loc[index[0]].values][index[1]].values.reshape(-1, 1)-ypred1
    
    lnclf2.fit(sensorDataMasked[index[1]].values.reshape(-1, 1), sensorDataMasked[index[0]].values.reshape(-1, 1))
    ypred2 = lnclf2.predict(sensorDataRed.loc[maskpd.loc[index[1]].values][index[1]].values.reshape(-1, 1))
    residuals2 = sensorDataRed.loc[maskpd.loc[index[1]].values][index[0]].values.reshape(-1, 1)-ypred2
    

In [ ]:
index[0], index[1]

In [ ]:
plt.plot(sensorDataRed.loc[maskpd.loc[index[0]].values][index[0]].values.reshape(-1, 1),residuals1, 'o', color='darkblue')
plt.title("Residual Plot")
plt.xlabel("Last sensor")
plt.ylabel("Residual")

In [ ]:
#import scipy
#import scipy.cluster.hierarchy as sch

In [ ]:
#plt.figure(figsize=(10, 7))  
#plt.title("Dendrograms") 
#pairwise_distances = sch.distance.pdist(sensorDataMasked.iloc[:,:-1].corr())
#linkage = sch.linkage(pairwise_distances, method='complete')
#dend = sch.dendrogram(linkage)

In [ ]:
#idx_to_cluster_array = sch.fcluster(linkage, 6,criterion='distance')

In [ ]:
#idx_to_cluster_array

In [ ]:
def cluster_corr(corr_array, inplace=False):
    """
    Rearranges the correlation matrix, corr_array, so that groups of highly 
    correlated variables are next to eachother 
    
    Parameters
    ----------
    corr_array : pandas.DataFrame or numpy.ndarray
        a NxN correlation matrix 
        
    Returns
    -------
    pandas.DataFrame or numpy.ndarray
        a NxN correlation matrix with the columns and rows rearranged
    """
    pairwise_distances = sch.distance.pdist(corr_array)
    linkage = sch.linkage(pairwise_distances, method='complete')
    cluster_distance_threshold = pairwise_distances.max()/2
    idx_to_cluster_array = sch.fcluster(linkage, cluster_distance_threshold, 
                                        criterion='distance')
    idx = np.argsort(idx_to_cluster_array)
    
    if not inplace:
        corr_array = corr_array.copy()
    
    if isinstance(corr_array, pd.DataFrame):
        return corr_array.iloc[idx, :].T.iloc[idx, :]
    return corr_array[idx, :][:, idx]

In [ ]:
#fig, ax = plt.subplots(figsize=(10,8)) 
#sns.heatmap(cluster_corr(sensorDataMasked.iloc[:,:-1].corr()), ax=ax)